In [1]:
import pandas as pd #For DataFrame Manipulation

import os #For accessing previously saved file

from geopy.geocoders import Nominatim #For accessing latitude and Longtitude

import folium #For Map Manipulation

import requests #For accessing url

import json #For json file manipulation

from pandas.io.json import json_normalize #for converting json file to dictionary

import numpy as np #Mathematical manipulation

from sklearn.cluster import KMeans #For clustering

import matplotlib.cm as cm #for plotting  on maps

import matplotlib.colors as colors #for color pallete

 The file 'final.csv' contains the final dataframe manipulated in a previous excercise

In [2]:
os.chdir('D:\\Capstone\Clustering')
final=pd.read_csv('final.csv')
final.head()

,Unnamed: 0,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [3]:
final.drop('Unnamed: 0',axis =1,inplace =True) 

In [4]:
Toronto=[]
for i in range(len(final)):
    if 'Toronto' in final.Borough[i]:
        Toronto.append(final.iloc[i,:])
tor=pd.DataFrame(Toronto)
tor.reset_index(drop=True,inplace=True)
tor.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
2,M4L,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


In [5]:
address = 'Toronto, CA' #Getting the coordinates for Toronto CA

geolocator = Nominatim(user_agent="Tr_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [6]:
map_Toronto = folium.Map(location=[latitude, longitude], zoom_start=10) #Creating a map of Toronto

for lat, lng, borough, neighborhood in zip(tor['Latitude'],tor['Longitude'],tor['Borough'],tor['Neighbourhood']): #Marking different Borough
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Toronto)  
    
map_Toronto

In [7]:
CLIENT_ID = 'O4QXRWP31S2P4IFTE50GVWNNSH0MDF24YYFOIANX4RS55JGT' # your Foursquare ID
CLIENT_SECRET = '1Q31LNOALLLPX0TGLTDNOIBUQP0E3UJTRSSW5RVKPDSDWG4M' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: O4QXRWP31S2P4IFTE50GVWNNSH0MDF24YYFOIANX4RS55JGT
CLIENT_SECRET:1Q31LNOALLLPX0TGLTDNOIBUQP0E3UJTRSSW5RVKPDSDWG4M


### Getting nearby venues for a single Borough

In [8]:
tor.loc[0, 'Neighbourhood']

'The Beaches'

In [9]:
neighborhood_latitude = tor.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = tor.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = tor.loc[0, 'Neighbourhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of The Beaches are 43.67635739999999, -79.2930312.


In [10]:

LIMIT = 100
radius =500
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)


In [11]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5caa17246a60712cef1e8c29'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'The Beaches',
  'headerFullLocation': 'The Beaches, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 4,
  'suggestedBounds': {'ne': {'lat': 43.680857404499996,
    'lng': -79.28682091449052},
   'sw': {'lat': 43.67185739549999, 'lng': -79.29924148550948}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4ad4c062f964a52011f820e3',
       'name': 'The Big Carrot Natural Food Market',
       'location': {'address': '125 Southwood Dr',
        'lat': 43.678879,
        'lng': -79.297734,
        'labeledLatLngs': [{'label': 'display',
          'lat

### Getting nearby venues for all the Borough

In [12]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [13]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,The Big Carrot Natural Food Market,Health Food Store,43.678879,-79.297734
1,Grover Pub and Grub,Pub,43.679181,-79.297215
2,Starbucks,Coffee Shop,43.678798,-79.298045
3,Upper Beaches,Neighborhood,43.680563,-79.292869


In [14]:

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius,
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [15]:
Toronto_venues = getNearbyVenues(names=tor['Neighbourhood'],
                                   latitudes=tor['Latitude'],
                                   longitudes=tor['Longitude']
                                  )

Toronto_venues.head()

The Beaches
The Danforth West, Riverdale
The Beaches West, India Bazaar
Studio District
Lawrence Park
Davisville North
North Toronto West
Davisville
Moore Park, Summerhill East
Deer Park, Forest Hill SE, Rathnelly, South Hill, Summerhill West
Rosedale
Cabbagetown, St. James Town
Church and Wellesley
Harbourfront, Regent Park
Ryerson, Garden District
St. James Town
Berczy Park
Central Bay Street
Adelaide, King, Richmond
Harbourfront East, Toronto Islands, Union Station
Design Exchange, Toronto Dominion Centre
Commerce Court, Victoria Hotel
Roselawn
Forest Hill North, Forest Hill West
The Annex, North Midtown, Yorkville
Harbord, University of Toronto
Chinatown, Grange Park, Kensington Market
CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara
Stn A PO Boxes 25 The Esplanade
First Canadian Place, Underground city
Christie
Dovercourt Village, Dufferin
Little Portugal, Trinity
Brockton, Exhibition Place, Parkdale Village
High Park, The 

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
1,The Beaches,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
2,The Beaches,43.676357,-79.293031,Starbucks,43.678798,-79.298045,Coffee Shop
3,The Beaches,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood
4,"The Danforth West, Riverdale",43.679557,-79.352188,Pantheon,43.677621,-79.351434,Greek Restaurant


In [16]:
Toronto_venues.shape

(1710, 7)

### Creating dummies for different venues

In [17]:
# one hot encoding
Tor_onehot = pd.get_dummies(Toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Tor_onehot['Neighborhood'] = Toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [Tor_onehot.columns[165]] + list(Tor_onehot.columns[:-165])#+list(Tor_onehot.columns[166:])
Tor_onehot = Tor_onehot[fixed_columns]
Tor_onehot.head()


,Neighborhood,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,...,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner
0,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"The Danforth West, Riverdale",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [18]:
Tor_onehot.shape


(1710, 76)

In [19]:
Tor_grouped = Tor_onehot.groupby('Neighborhood').mean().reset_index()
Tor_grouped

,Neighborhood,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,...,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner
0,"Adelaide, King, Richmond",0.01,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.010000,0.010000,0.010000,0.000000,0.00,0.000000
1,Berczy Park,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.018182,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.018182
2,"Brockton, Exhibition Place, Parkdale Village",0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
3,Business Reply Mail Processing Centre 969 Eastern,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
4,"CN Tower, Bathurst Quay, Island airport, Harbo...",0.00,0.000000,0.000000,0.076923,0.076923,0.076923,0.153846,0.153846,0.153846,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
5,"Cabbagetown, St. James Town",0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.023256,0.000000,0.000000,0.00,0.023256
6,Central Bay Street,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.011905,0.011905,0.00,0.011905
7,"Chinatown, Grange Park, Kensington Market",0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.020000,0.02,0.000000
8,Christie,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.062500
9,Church and Wellesley,0.00,0.011236,0.011236,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.011236,0.000000,0.000000,0.011236,0.000000,0.000000,0.000000,0.00,0.011236


In [20]:
num_top_venues = 3  
for hood in Tor_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = Tor_grouped[Tor_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide, King, Richmond----
         venue  freq
0  Coffee Shop  0.06
1          Bar  0.04
2         Café  0.04


----Berczy Park----
          venue  freq
0   Coffee Shop  0.07
1  Cocktail Bar  0.05
2          Café  0.04


----Brockton, Exhibition Place, Parkdale Village----
            venue  freq
0  Breakfast Spot   0.1
1            Café   0.1
2     Coffee Shop   0.1


----Business Reply Mail Processing Centre 969 Eastern----
           venue  freq
0        Brewery  0.06
1  Auto Workshop  0.06
2  Burrito Place  0.06


----CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara----
              venue  freq
0    Airport Lounge  0.15
1   Airport Service  0.15
2  Airport Terminal  0.15


----Cabbagetown, St. James Town----
         venue  freq
0  Coffee Shop  0.09
1         Café  0.05
2       Bakery  0.05


----Central Bay Street----
             venue  freq
0      Coffee Shop  0.15
1  Bubble Tea Shop  0.04
2     Burger Joint  0.0

In [21]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [22]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = Tor_grouped['Neighborhood']

for ind in np.arange(Tor_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Tor_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",Coffee Shop,Bar,Café,American Restaurant,Burger Joint,Bakery,Concert Hall,Breakfast Spot,Asian Restaurant,Cosmetics Shop
1,Berczy Park,Coffee Shop,Cocktail Bar,Bakery,Café,Cheese Shop,Diner,Art Gallery,Beer Bar,Breakfast Spot,Beach
2,"Brockton, Exhibition Place, Parkdale Village",Coffee Shop,Breakfast Spot,Café,Bar,Caribbean Restaurant,Burrito Place,Climbing Gym,Convenience Store,Diner,Baby Store
3,Business Reply Mail Processing Centre 969 Eastern,Brewery,Burrito Place,Comic Shop,Auto Workshop,Baseball Stadium,Baby Store,Bagel Shop,Bakery,Bank,Bar
4,"CN Tower, Bathurst Quay, Island airport, Harbo...",Airport Lounge,Airport Service,Airport Terminal,Boat or Ferry,Airport,Airport Food Court,Airport Gate,Beer Bar,Bank,Bar


## Clustering

In [23]:
# set number of clusters
kclusters = 5

Tor_grouped_clustering = Tor_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Tor_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 1, 1, 0, 3, 1, 1, 2, 1, 0])

In [24]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

Tor_merged = tor

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
Tor_merged = Tor_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighbourhood')

Tor_merged.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,East Toronto,The Beaches,43.676357,-79.293031,1,Coffee Shop,Basketball Stadium,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Beach
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188,1,Coffee Shop,Diner,Cosmetics Shop,American Restaurant,Bakery,Bookstore,Bubble Tea Shop,Café,Caribbean Restaurant,Brewery
2,M4L,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572,0,Brewery,Burrito Place,Burger Joint,Coffee Shop,Airport,Beer Bar,Bakery,Bank,Bar,Baseball Stadium
3,M4M,East Toronto,Studio District,43.659526,-79.340923,1,Café,Coffee Shop,Bakery,American Restaurant,Brewery,Convenience Store,Bar,Bank,Cheese Shop,Chinese Restaurant
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,4,Bus Line,Diner,Beach,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium


In [25]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Tor_merged['Latitude'], Tor_merged['Longitude'], Tor_merged['Neighbourhood'], Tor_merged['Cluster Labels']):
    label = folium.Popup(str(poi), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Printing the clusters seperate

In [26]:
Tor_merged.loc[Tor_merged['Cluster Labels'] == 0, Tor_merged.columns[[1] + list(range(5, 9))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue
2,East Toronto,0,Brewery,Burrito Place,Burger Joint
5,Central Toronto,0,Breakfast Spot,Asian Restaurant,Dance Studio
7,Central Toronto,0,Dessert Shop,Coffee Shop,Café
8,Central Toronto,0,Diner,Auto Workshop,Baby Store
10,Downtown Toronto,0,Diner,Auto Workshop,Baby Store
12,Downtown Toronto,0,Coffee Shop,Burger Joint,Café
22,Central Toronto,0,Diner,Auto Workshop,Baby Store
23,Central Toronto,0,Diner,Auto Workshop,Baby Store
35,West Toronto,0,Breakfast Spot,Bookstore,Dessert Shop
37,East Toronto,0,Brewery,Burrito Place,Comic Shop


In [27]:
Tor_merged.loc[Tor_merged['Cluster Labels'] == 1, Tor_merged.columns[[1] + list(range(5, 9))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue
0,East Toronto,1,Coffee Shop,Basketball Stadium,BBQ Joint
1,East Toronto,1,Coffee Shop,Diner,Cosmetics Shop
3,East Toronto,1,Café,Coffee Shop,Bakery
6,Central Toronto,1,Coffee Shop,Clothing Store,Chinese Restaurant
9,Central Toronto,1,Coffee Shop,Convenience Store,Bagel Shop
11,Downtown Toronto,1,Coffee Shop,Café,Bakery
13,Downtown Toronto,1,Coffee Shop,Bakery,Café
14,Downtown Toronto,1,Coffee Shop,Clothing Store,Café
15,Downtown Toronto,1,Coffee Shop,Café,Breakfast Spot
16,Downtown Toronto,1,Coffee Shop,Cocktail Bar,Bakery


In [28]:
Tor_merged.loc[Tor_merged['Cluster Labels'] == 2, Tor_merged.columns[[1] + list(range(5, 9))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue
25,Downtown Toronto,2,Café,Bookstore,Bar
26,Downtown Toronto,2,Café,Bar,Bakery
31,West Toronto,2,Bakery,Brewery,Bank
32,West Toronto,2,Bar,Coffee Shop,Asian Restaurant
34,West Toronto,2,Bar,Café,Diner


In [29]:
Tor_merged.loc[Tor_merged['Cluster Labels'] == 3, Tor_merged.columns[[1] + list(range(5, 9))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue
27,Downtown Toronto,3,Airport Lounge,Airport Service,Airport Terminal


In [30]:
Tor_merged.loc[Tor_merged['Cluster Labels'] == 4, Tor_merged.columns[[1] + list(range(5, 9))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue
4,Central Toronto,4,Bus Line,Diner,Beach
